In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import output

In [ ]:
card_map = {
    -2: "Two-Eyed Jack",
    -1: "One-Eyed Jack",
    1: "10c1",
    2: "9c1",
    3: "8c1",
    4: "7c1",
    5: "7h1",
    6: "8h1",
    7: "9h1",
    8: "10h1",
    9: "10d1",
    10: "Kd1",
    11: "6c1",
    12: "5c1",
    13: "4c1",
    14: "4h1",
    15: "5h1",
    16: "6h1",
    17: "Ks1",
    18: "10s1",
    19: "9d1",
    20: "6d1",
    21: "Qd1",
    22: "3c1",
    23: "2c1",
    24: "2h1",
    25: "3h1",
    26: "Qs1",
    27: "6s1",
    28: "9s1",
    29: "8d1",
    30: "5d1",
    31: "3d1",
    32: "Qc1",
    33: "Ac1",
    34: "Ah1",
    35: "Qh1",
    36: "3s1",
    37: "5s1",
    38: "8s1",
    39: "7d1",
    40: "4d1",
    41: "2d1",
    42: "Ad1",
    43: "Kc1",
    44: "Kh1",
    45: "As1",
    46: "2s1",
    47: "4s1",
    48: "7s1",
    49: "7s2",
    50: "4s2",
    51: "2s2",
    52: "As2",
    53: "Kh2",
    54: "Kc2",
    55: "Ad2",
    56: "2d2",
    57: "4d2",
    58: "7d2",
    59: "8s2",
    60: "5s2",
    61: "3s2",
    62: "Qh2",
    63: "Ah2",
    64: "Ac2",
    65: "Qc2",
    66: "3d2",
    67: "5d2",
    68: "8d2",
    69: "9s2",
    70: "6s2",
    71: "Qs2",
    72: "3h2",
    73: "2h2",
    74: "2c2",
    75: "3c2",
    76: "Qd2",
    77: "6d2",
    78: "9d2",
    79: "10s2",
    80: "Ks2",
    81: "6h2",
    82: "5h2",
    83: "4h2",
    84: "4c2",
    85: "5c2",
    86: "6c2",
    87: "Kd2",
    88: "10d2",
    89: "10h2",
    90: "9h2",
    91: "8h2",
    92: "7h2",
    93: "7c2",
    94: "8c2",
    95: "9c2",
    96: "10c2",
}

In [ ]:
idx_transform = [(i, j) for i in range(10) for j in range(10)
                       if (i, j) not in [(0, 0), (0, 9), (9, 0), (9, 9)]]

In [ ]:
def getScore(card, curr_board):
  box_size = 4
  x = idx_transform[card - 1][0]
  y = idx_transform[card - 1][1]
  half_size = box_size // 2

  # Define the boundaries
  x_start = max(0, x - half_size)
  x_end = min(curr_board.shape[0], x + half_size + 1)

  y_start = max(0, y - half_size)
  y_end = min(curr_board.shape[1], y + half_size + 1)

  # Sum when > 0
  box = curr_board[x_start:x_end, y_start:y_end]
  box_sum = np.sum(box[box > 0])
  return box_sum

In [ ]:
def getBestCard(hand, curr_board):
  score = 0
  best_card = 0

  for card in hand:
    if card == -1 or card == -2:
      continue
    x1 = idx_transform[card - 1][0]
    y1 = idx_transform[card - 1][1]
    x2 = idx_transform[97-card - 1][0]
    y2 = idx_transform[97-card - 1][1]

    if board[x1][y1] == .33:
      if getScore(card, curr_board) > score:
        score = getScore(card, curr_board)
        best_card = card

    if board[x2][y2] == .33:
      if getScore(97-card, curr_board) > score:
        score = getScore(97-card, curr_board)
        best_card = 97-card

  return best_card

In [ ]:
def find_one_away_sequences(board, target_value):
    rows, cols = board.shape
    sequences = []

    # Check horizontal sequences
    for i in range(rows):
        for j in range(cols - 4):
            window = board[i, j:j + 5]
            if valid_one_away(window, target_value):
                sequences.append((i, j + np.where(window == 0)[0][0]))

    # Check vertical sequences
    for i in range(rows - 4):
        for j in range(cols):
            window = board[i:i + 5, j]
            if valid_one_away(window, target_value):
                sequences.append((i + np.where(window == 0)[0][0], j))

    # Check diagonal (top-left to bottom-right) sequences
    for i in range(rows - 4):
        for j in range(cols - 4):
            window = [board[i + k, j + k] for k in range(5)]
            if valid_one_away(window, target_value):
                missing_index = np.where(np.array(window) == 0)[0][0]
                sequences.append((i + missing_index, j + missing_index))

    # Check diagonal (top-right to bottom-left) sequences
    for i in range(rows - 4):
        for j in range(4, cols):
            window = [board[i + k, j - k] for k in range(5)]
            if valid_one_away(window, target_value):
                missing_index = np.where(np.array(window) == 0)[0][0]
                sequences.append((i + missing_index, j - missing_index))

    return sequences

def valid_one_away(window, target_value):
    """Check if the window has 4 of target_value, 0 or 1 zero, and at most one -2."""
    window = np.array(window)
    num_target = np.sum(window == target_value)
    num_zeros = np.sum(window == 0)
    num_neg_twos = np.sum(window == -2)

    # Valid if 4 target_values, 1 zero, and at most 1 occurrence of -2
    return num_target == 4 and num_zeros == 1 and num_neg_twos <= 1

In [ ]:
def check_and_update_sequences(board):
    rows, cols = board.shape
    directions = [
        (0, 1),   # Horizontal →
        (1, 0),   # Vertical ↓
        (1, 1),   # Diagonal ↘
        (1, -1)   # Diagonal ↙
    ]

    for x in range(rows):
        for y in range(cols):
            if board[x, y] == 1:
                for dx, dy in directions:
                    if check_sequence(board, x, y, dx, dy):
                        update_sequence(board, x, y, dx, dy)

def check_sequence(board, x, y, dx, dy):
    """Check if there's a sequence of 5 starting at (x, y) in direction (dx, dy)."""
    rows, cols = board.shape
    for i in range(5):
        nx, ny = x + i * dx, y + i * dy
        if nx < 0 or nx >= rows or ny < 0 or ny >= cols or board[nx, ny] != 1:
            return False
    return True

def update_sequence(board, x, y, dx, dy):
    """Turn the sequence of 5 ones into -1s starting at (x, y)."""
    for i in range(5):
        nx, ny = x + i * dx, y + i * dy
        board[nx, ny] = -2

In [ ]:
# RUN TO START NEW GAME

# Initialize board, wild spaces
board = np.zeros((10,10))
board[0][0], board[0][9], board[9][0], board[9][9] = 1, 1, 1, 1

# Initialize hand
## One Eyed Jack: -1
## Two Eyed Jack: -2

hand = np.zeros(7, dtype=int)
for i in range(7):
  val = input("Enter Card: ")
  key = next(k for k, v in card_map.items() if v == val)
  hand[i] = key

# Set the board
for card in hand:
  if card == -1 or card == -2:
    continue
  x1 = idx_transform[card - 1][0]
  y1 = idx_transform[card - 1][1]
  x2 = idx_transform[97-card - 1][0]
  y2 = idx_transform[97-card - 1][1]
  if board[x1][y1] == 0:
    board[x1][y1] = .33
  if board[x2][y2] == 0:
    board[x2][y2] = .33

print(board)

In [ ]:
## Then run this

opp_move_y_n = input("If opponent first move, enter 'y', else 'n': ")

if opp_move_y_n == 'y':
  opp_move_first = True
else:
  opp_move_first = False

while True:

  if not opp_move_first:
    own_sequence = find_one_away_sequences(board, 1)

    if len(own_sequence) > 0 and np.isin(-2, hand):
      print("Play Sequence With Jack at: ", own_sequence[0])
      board[own_sequence[0]] = 1
      index_to_delete = np.where(hand == -2)[0]
      if index_to_delete.size > 0:
        hand = np.delete(hand, index_to_delete[0])

    else:
      # Get best move, update board + hand
      best_move = getBestCard(hand, board)
      print("Play: ", card_map[best_move])
      board[idx_transform[best_move - 1]] = 1

      if best_move > 48:
        index_to_delete = np.where(hand == (97 - best_move))[0]
      else:
        index_to_delete = np.where(hand == best_move)[0]

      # Check if the value exists before deleting
      if index_to_delete.size > 0:
        hand = np.delete(hand, index_to_delete[0])
      elif best_move > 48:
        board[idx_transform[best_move - 1]] = 0
      else:
        board[idx_transform[97 - best_move - 1]] = 0

    check_and_update_sequences(board)
    print(board)

    # Get new card, update board + hand
    new_card = input("New Card: ")
    new_key = next(k for k, v in card_map.items() if v == new_card)
    if new_key != -1 and new_key != -2:
      if board[idx_transform[new_key - 1]] == 0:
        board[idx_transform[new_key - 1]] = .33
      if board[idx_transform[97-new_key - 1]] == 0:
        board[idx_transform[97-new_key - 1]] = .33
    hand = np.append(hand, new_key)

  opp_move_first = False

  # Get opponent move, update board
  opp_played = input("Opponent Played: ")
  if opp_played == "One-Eyed Jack":
    where = input("Where?")
    opp_key = next(k for k, v in card_map.items() if v == where)
    board[idx_transform[opp_key - 1]] = 0
  else:
    opp_key = next(k for k, v in card_map.items() if v == opp_played)
    board[idx_transform[opp_key - 1]] = -1
  print(board)

  output.clear()